# 배터리 데이터 사이클러 분류 및 로딩 시스템

이 노트북은 배터리 충방전 데이터 경로를 자동으로 분석하여 **토요(Toyo)** 또는 **PNE** 사이클러로 분류하고, 데이터를 로딩합니다.

**분류 기준**:
- **PNE**: Pattern 폴더가 존재하는 경로
- **Toyo**: Pattern 폴더가 없는 경로

In [ ]:
# 필수 라이브러리 import
import os
import re
import glob
from pathlib import Path
from typing import List, Dict, Optional, Tuple
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

## 유틸리티 함수

In [ ]:
def check_cycler(path: str) -> str:
    """
    경로의 사이클러 타입을 식별합니다.
    
    Parameters:
    -----------
    path : str
        확인할 데이터 경로
    
    Returns:
    --------
    str
        'PNE' 또는 'Toyo' 또는 'Unknown'
    """
    if not os.path.exists(path):
        return "Unknown (경로 없음)"
    
    # Pattern 폴더 존재 여부로 PNE와 Toyo 구분
    pattern_path = os.path.join(path, "Pattern")
    
    if os.path.isdir(pattern_path):
        return "PNE"
    else:
        return "Toyo"


def extract_capacity_from_path(path: str) -> Optional[float]:
    """
    경로명에서 용량 정보를 추출합니다.
    
    Parameters:
    -----------
    path : str
        분석할 경로
    
    Returns:
    --------
    float or None
        추출된 용량(mAh), 없으면 None
    """
    # 정규 표현식으로 mAh 추출
    match = re.search(r'(\d+([\-.]\d+)?)mAh', path)
    if match:
        capacity_str = match.group(1).replace('-', '.')
        return float(capacity_str)
    return None


def get_directory_info(path: str) -> Dict[str, any]:
    """
    디렉토리의 메타 정보를 추출합니다.
    
    Parameters:
    -----------
    path : str
        분석할 디렉토리 경로
    
    Returns:
    --------
    dict
        디렉토리 정보 (하위 폴더 수, 파일 수 등)
    """
    info = {
        'exists': False,
        'subdirs': 0,
        'files': 0,
        'has_pattern_folder': False,
        'file_types': set()
    }
    
    if not os.path.exists(path):
        return info
    
    info['exists'] = True
    
    try:
        items = os.listdir(path)
        
        for item in items:
            item_path = os.path.join(path, item)
            
            if os.path.isdir(item_path):
                info['subdirs'] += 1
                if item.lower() == 'pattern':
                    info['has_pattern_folder'] = True
            else:
                info['files'] += 1
                # 파일 확장자 추출
                _, ext = os.path.splitext(item)
                if ext:
                    info['file_types'].add(ext.lower())
    
    except PermissionError:
        info['error'] = '권한 없음'
    except Exception as e:
        info['error'] = str(e)
    
    return info


def classify_battery_paths(path_list: List[str]) -> pd.DataFrame:
    """
    여러 배터리 데이터 경로를 일괄 분류합니다.
    
    Parameters:
    -----------
    path_list : List[str]
        분류할 경로 리스트
    
    Returns:
    --------
    pd.DataFrame
        분류 결과가 담긴 데이터프레임
    """
    results = []
    
    for path in path_list:
        # 기본 정보
        cycler_type = check_cycler(path)
        dir_info = get_directory_info(path)
        capacity = extract_capacity_from_path(path)
        
        # 경로의 마지막 부분만 추출 (가독성)
        path_name = os.path.basename(path)
        
        results.append({
            '경로': path,
            '폴더명': path_name,
            '사이클러': cycler_type,
            '용량(mAh)': capacity if capacity else '-',
            '하위폴더수': dir_info['subdirs'],
            '파일수': dir_info['files'],
            'Pattern폴더': '있음' if dir_info['has_pattern_folder'] else '없음',
            '존재여부': '존재' if dir_info['exists'] else '없음'
        })
    
    df = pd.DataFrame(results)
    return df

## PNE 데이터 로딩 함수

In [ ]:
def find_restore_folders(pne_path: str) -> List[str]:
    """
    PNE 경로에서 Restore 폴더를 찾습니다.
    
    Parameters:
    -----------
    pne_path : str
        PNE 데이터 경로
    
    Returns:
    --------
    List[str]
        Restore 폴더 경로 리스트
    """
    restore_folders = []
    
    # M##Ch###[###] 형식의 채널 폴더 찾기
    for item in os.listdir(pne_path):
        item_path = os.path.join(pne_path, item)
        if os.path.isdir(item_path) and re.match(r'M\d+Ch\d+\[\d+\]', item):
            # Restore 폴더 확인
            restore_path = os.path.join(item_path, 'Restore')
            if os.path.isdir(restore_path):
                restore_folders.append(restore_path)
    
    return sorted(restore_folders)


def load_pne_restore_data(restore_path: str, max_files: Optional[int] = None) -> pd.DataFrame:
    """
    PNE Restore 폴더에서 시계열 프로파일 데이터를 로딩합니다.
    SaveData 파일들을 순서대로 읽어서 하나의 DataFrame으로 합칩니다.
    
    Parameters:
    -----------
    restore_path : str
        Restore 폴더 경로
    max_files : int, optional
        읽을 최대 파일 개수 (테스트용)
    
    Returns:
    --------
    pd.DataFrame
        합쳐진 시계열 프로파일 데이터
    """
    # SaveData 파일들 찾기 (ch##_SaveData####.csv 형식)
    savedata_pattern = os.path.join(restore_path, '*_SaveData*.csv')
    savedata_files = sorted(glob.glob(savedata_pattern))
    
    if not savedata_files:
        print(f"경고: {restore_path}에서 SaveData 파일을 찾을 수 없습니다.")
        return pd.DataFrame()
    
    # 파일명에서 숫자 추출하여 정렬 (ch09_SaveData0001.csv -> 1)
    def extract_file_number(filepath):
        match = re.search(r'SaveData(\d+)\.csv', filepath)
        return int(match.group(1)) if match else 0
    
    savedata_files = sorted(savedata_files, key=extract_file_number)
    
    # 테스트용: 파일 개수 제한
    if max_files:
        savedata_files = savedata_files[:max_files]
    
    print(f"총 {len(savedata_files)}개의 SaveData 파일 발견")
    print(f"첫 파일: {os.path.basename(savedata_files[0])}")
    print(f"마지막 파일: {os.path.basename(savedata_files[-1])}")
    
    # 시계열 순서대로 파일 읽어서 합치기
    dfs = []
    for i, file_path in enumerate(savedata_files):
        try:
            # CSV 파일 읽기 (헤더 없음, 콤마 구분)
            df = pd.read_csv(file_path, header=None)
            dfs.append(df)
            
            # 진행 상황 출력 (10개마다)
            if (i + 1) % 10 == 0:
                print(f"  진행: {i + 1}/{len(savedata_files)} 파일 로딩 완료")
        
        except Exception as e:
            print(f"경고: {os.path.basename(file_path)} 로딩 실패 - {e}")
            continue
    
    if not dfs:
        print("오류: 로딩된 데이터가 없습니다.")
        return pd.DataFrame()
    
    # 모든 DataFrame을 시계열 순서로 합치기
    combined_df = pd.concat(dfs, axis=0, ignore_index=True)
    
    print(f"\n✅ 데이터 로딩 완료: {len(combined_df):,}행 x {len(combined_df.columns)}열")
    
    return combined_df


def get_pne_channel_data(pne_path: str, channel_index: int = 0, max_files: Optional[int] = None) -> Dict:
    """
    PNE 경로에서 특정 채널의 데이터를 로딩합니다.
    
    Parameters:
    -----------
    pne_path : str
        PNE 데이터 경로
    channel_index : int
        채널 인덱스 (0부터 시작)
    max_files : int, optional
        읽을 최대 파일 개수 (테스트용)
    
    Returns:
    --------
    dict
        {'channel_name': str, 'restore_path': str, 'data': DataFrame}
    """
    # Restore 폴더 찾기
    restore_folders = find_restore_folders(pne_path)
    
    if not restore_folders:
        print(f"경고: {pne_path}에서 Restore 폴더를 찾을 수 없습니다.")
        return {'channel_name': None, 'restore_path': None, 'data': pd.DataFrame()}
    
    print(f"\n발견된 Restore 폴더: {len(restore_folders)}개")
    for i, folder in enumerate(restore_folders):
        print(f"  [{i}] {os.path.dirname(folder).split(os.sep)[-1]}")
    
    if channel_index >= len(restore_folders):
        print(f"오류: 채널 인덱스 {channel_index}가 범위를 벗어났습니다.")
        return {'channel_name': None, 'restore_path': None, 'data': pd.DataFrame()}
    
    # 선택된 채널의 데이터 로딩
    selected_restore = restore_folders[channel_index]
    channel_name = os.path.dirname(selected_restore).split(os.sep)[-1]
    
    print(f"\n선택된 채널: {channel_name}")
    print(f"Restore 경로: {selected_restore}")
    print("\n데이터 로딩 중...")
    
    data = load_pne_restore_data(selected_restore, max_files=max_files)
    
    return {
        'channel_name': channel_name,
        'restore_path': selected_restore,
        'data': data
    }

## 경로 입력

분석할 배터리 데이터 경로를 입력합니다.

In [ ]:
# 경로 리스트 (기본값: 기존 경로들)
paths = [
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\250219_250319_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 101-200cyc",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\250304_250404_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 201-300cyc",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\250317_251231_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 301-400cyc",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\A1_MP1_4500mAh_T23_1",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\A1_MP1_4500mAh_T23_2",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\A1_MP1_4500mAh_T23_3",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Dateset_A1_Gen4 2C ATL MP2 [45V 4470mAh] [23] blk2",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Gen4 2C ATL MP2 [45V 4470mAh] [23] blk7 - 240131",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\M1 ATL [45V 4175mAh]",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Q7M Inner ATL_45V 1689mAh BLK1 20EA [23] - 250304",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Q7M Main ATL [45V_1680mAh][23] blk7 20ea - 250228",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Q7M Sub ATL [45v 2068mAh] [23] - 250219r"
]

print(f"총 {len(paths)}개 경로가 입력되었습니다.")

## 사이클러 자동 분류

In [ ]:
# 경로 분류 실행
results_df = classify_battery_paths(paths)

# 결과 출력
print("\n=== 배터리 데이터 사이클러 분류 결과 ===")
print(f"\n총 분석 경로: {len(results_df)}개")
print(f"\n사이클러 타입별 통계:")
print(results_df['사이클러'].value_counts())

# 전체 결과 테이블 (경로 제외 - 가독성)
display_df = results_df.drop(columns=['경로'])
display(display_df)

## PNE 데이터 로딩 예제

PNE 경로에서 Restore 폴더의 시계열 프로파일 데이터를 로딩합니다.

In [ ]:
# PNE 경로 필터링
pne_paths = results_df[results_df['사이클러'] == 'PNE']['경로'].tolist()

if pne_paths:
    print(f"발견된 PNE 경로: {len(pne_paths)}개\n")
    for i, path in enumerate(pne_paths):
        print(f"[{i}] {os.path.basename(path)}")
    
    # 첫 번째 PNE 경로 선택
    selected_pne_path = pne_paths[0]
    print(f"\n선택된 경로: {os.path.basename(selected_pne_path)}")
else:
    print("PNE 경로가 없습니다.")
    selected_pne_path = None

In [ ]:
# PNE 데이터 로딩 (테스트: 처음 5개 파일만)
if selected_pne_path:
    channel_data = get_pne_channel_data(
        selected_pne_path, 
        channel_index=0,  # 첫 번째 채널
        max_files=5       # 테스트용: 5개 파일만 (None으로 설정하면 전체 로딩)
    )
    
    # 로딩된 데이터 확인
    if not channel_data['data'].empty:
        print(f"\n=== 로딩된 데이터 정보 ===")
        print(f"채널명: {channel_data['channel_name']}")
        print(f"데이터 shape: {channel_data['data'].shape}")
        print(f"\n처음 10행:")
        display(channel_data['data'].head(10))
        print(f"\n마지막 10행:")
        display(channel_data['data'].tail(10))
        
        # 데이터 통계
        print(f"\n=== 데이터 통계 ===")
        display(channel_data['data'].describe())
else:
    print("PNE 경로가 선택되지 않았습니다.")

## 요약

### 완료된 작업
✅ 배터리 데이터 경로 자동 분류 (Toyo/PNE)  
✅ 디렉토리 메타 정보 추출  
✅ 용량 정보 자동 추출  
✅ **PNE Restore 폴더 시계열 데이터 로딩**  
✅ **SaveData 파일 순서대로 읽어 합치기**  

### 사용법

**1. 사이클러 분류만 하기**:
- "경로 입력" 및 "사이클러 자동 분류" 셀 실행

**2. PNE 데이터 로딩**:
- "PNE 데이터 로딩 예제" 섹션 실행
- `max_files` 파라미터로 로딩할 파일 수 조절 (테스트: 5, 전체: None)
- `channel_index`로 원하는 채널 선택

**3. 다른 PNE 경로 선택**:
```python
selected_pne_path = pne_paths[1]  # 두 번째 PNE 경로
```